第一种方法是结构化的方法。首先思考结构化的方法。如果信息整理难度过大就走统计学的路。

第二种方法是统计学的方法。

第三种方法是手动导入。至少对于我个人看过的番剧来说，手动导入是可以实现的。另外，如果有社区的话，实现起来也非常容易。但是，这就不能体现技术上的geek感。




注：如果采用selenium之类的，可以使用这个

```js
$('.mic-evaluate')[0].querySelector('span').innerText

// or 
document.querySelector('.mic-evaluate').querySelector('span').innerText


`御坂美琴：佐藤利奈
佐天泪子：伊藤加奈惠
初春饰利：丰崎爱生
白井黑子：新井里美
麦野沈利：小清水亚美
绢旗最爱：赤﨑千夏
一方通行：岡本信彦
芙兰达·赛维伦：内田真礼
布束砥信：葉山いくみ
`
```

网址的问题

https://www.bilibili.com/bangumi/media/md427

关键是找到md后面的数字的列表，之后就好办了。

不管怎样，先验证爬虫吧。

In [50]:
import requests
from bs4 import BeautifulSoup
import re

In [51]:
H = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"}

res = requests.get('https://www.bilibili.com/bangumi/media/md427', headers=H)

In [13]:
with open("a.html", 'w', encoding='utf8') as f:
    f.write(res.text)

In [27]:
# 方法1 "actors": "..."

pattern = r'"actors"[^"]*"(.+?)"'
re.findall(pattern, res.text)[0]

'御坂美琴：佐藤利奈\\n佐天泪子：伊藤加奈惠\\n初春饰利：丰崎爱生\\n白井黑子：新井里美\\n麦野沈利：小清水亚美\\n绢旗最爱：赤﨑千夏\\n一方通行：岡本信彦\\n芙兰达·赛维伦：内田真礼\\n布束砥信：葉山いくみ\\n'

接下来就是找url的环节。



In [28]:
res = requests.get('https://www.bilibili.com/bangumi/play/ss3398', headers=H)

In [30]:
with open('b.html','w',encoding='utf8') as f:
    f.write(res.text)

In [32]:
s = BeautifulSoup(res.text)

In [47]:
atag = s.select_one(".media-title")

print("https:"+atag.attrs['href'])
print(atag.text)

https://www.bilibili.com/bangumi/media/md3398/
冰菓


# 正片开始

In [58]:
# 获取全部番剧列表，按照排行榜排序

def compose_url(page, page_size):
    tmpl = "https://api.bilibili.com/pgc/season/index/result?season_version=-1&area=-1&is_finish=-1&copyright=-1&season_status=-1&season_month=-1&year=-1&style_id=-1&order=3&st=1&sort=0&page={0}&season_type=1&pagesize={1}&type=1"
    return tmpl.format(page, page_size)

compose_url(10,100)

'https://api.bilibili.com/pgc/season/index/result?season_version=-1&area=-1&is_finish=-1&copyright=-1&season_status=-1&season_month=-1&year=-1&style_id=-1&order=3&st=1&sort=0&page=10&season_type=1&pagesize=100&type=1'

In [59]:
huge_url = compose_url(1, 10000)

In [60]:
res = requests.get(huge_url, headers=H)

In [69]:
with open('data/1-all-anime-list.json', 'w', encoding='utf8') as f:
    f.write(res.text)

In [63]:
obj = res.json()

In [65]:
obj['code']

0

In [67]:
obj['data']['list'].__len__()

3357

In [70]:
anime_urls = [p['link'] for p in obj['data']['list']]

In [72]:
anime_urls[:10]

['https://www.bilibili.com/bangumi/play/ss26801',
 'https://www.bilibili.com/bangumi/play/ss34430',
 'https://www.bilibili.com/bangumi/play/ss36174',
 'https://www.bilibili.com/bangumi/play/ss36429',
 'https://www.bilibili.com/bangumi/play/ss38221',
 'https://www.bilibili.com/bangumi/play/ss36170',
 'https://www.bilibili.com/bangumi/play/ss34004',
 'https://www.bilibili.com/bangumi/play/ss24588',
 'https://www.bilibili.com/bangumi/play/ss33802',
 'https://www.bilibili.com/bangumi/play/ss32982']

In [74]:
# get title and cv actor list of 1 sample

sample_url = anime_urls[0]
res = requests.get(sample_url, headers=H)
s = BeautifulSoup(res.text)

atag = s.select_one(".media-title")

print("https:"+atag.attrs['href'])
print(atag.text)

https://www.bilibili.com/bangumi/media/md22718131/
鬼灭之刃


In [75]:
import hashlib
def sha256(s):  
    h = hashlib.sha256()   
    h.update(s.encode('utf8'))   
    b = h.hexdigest().lower() 
    return b

In [84]:
# download all webpages

# download 1 webpage

def compose_webpage_store(url):
    return f'data/2-all/{sha256(url)}.html'

url = anime_urls[0]
res = requests.get(sample_url, headers=H)

with open(compose_webpage_store(url), 'w', encoding='utf8') as f:
    f.write(res.text)